<a href="https://colab.research.google.com/github/Limteckping45/COS30019Assignment2/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
from PIL import Image
import matplotlib.pyplot as plt

!git clone "https://github.com/Limteckping45/COS30019Assignment2"

train_data_dir = "/content/COS30019Assignment2/training_dataset"
test_data_dir = "/content/COS30019Assignment2/testing_dataset"
validation_data_dir = "/content/COS30019Assignment2/validation_dataset_1"

# Define image dimensions
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 32

# Pre-process and augment data
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Load pre-trained DenseNet121 model without top layers
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=input_shape)

# Extract features from train, validation, and test data
def extract_features(generator, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 1024))  # DenseNet121 output shape
    labels = np.zeros(shape=(sample_count, generator.num_classes))

    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = base_model.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break

    return features, labels

train_features, train_labels = extract_features(train_generator, train_generator.samples)
validation_features, validation_labels = extract_features(validation_generator, validation_generator.samples)
test_features, test_labels = extract_features(test_generator, test_generator.samples)

# Flatten extracted features
train_features = np.reshape(train_features, (train_generator.samples, 7 * 7 * 1024))
validation_features = np.reshape(validation_features, (validation_generator.samples, 7 * 7 * 1024))
test_features = np.reshape(test_features, (test_generator.samples, 7 * 7 * 1024))

# Convert labels from one-hot encoding to label encoding
train_labels = np.argmax(train_labels, axis=1)
validation_features = np.reshape(validation_features, (validation_generator.samples, 7 * 7 * 1024))
test_labels = np.argmax(test_labels, axis=1)

# Train KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(train_features, train_labels)

# Save the KNN model
joblib.dump(knn_model, 'knn_animal_breed_classifier.pkl')

Cloning into 'COS30019Assignment2'...
remote: Enumerating objects: 3592, done.
remote: Counting objects: 100% (1198/1198), done.
remote: Compressing objects: 100% (1164/1164), done.
remote: Total 3592 (delta 35), reused 1194 (delta 34), pack-reused 2394
Receiving objects: 100% (3592/3592), 409.00 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (3281/3281), done.
Found 1920 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.
Found 120 images belonging to 12 classes.
1/1 [==============================] - 3s 3s/step


['knn_animal_breed_classifier.pkl']

In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.7 

In [3]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import joblib
import os

# Clone the GitHub repository
if not os.path.exists("COS30019Assignment2"):
    os.system("git clone https://github.com/Limteckping45/COS30019Assignment2")

# Paths to the model files in the cloned repository
transfer_learning_model_path = "/content/COS30019Assignment2/TransferLearning_Model.h5"
cnn_model_path = "/content/COS30019Assignment2/CNN_Model.h5"

# Load the models
transfer_learning_model = load_model(transfer_learning_model_path)
cnn_model = load_model(cnn_model_path)
knn_model = joblib.load('knn_animal_breed_classifier.pkl')

# Define labels (assuming these are the labels used during training)
labels = [
    'Bengal', 'Boxer', 'Chihuahua', 'Havanese', 'Japanese Chin',
    'Maine Coon', 'Persian', 'Pug', 'Ragdoll', 'Russian Blue',
    'Saint Bernard', 'Siamese'
]

# Base model for KNN feature extraction (DenseNet121)
base_model = tf.keras.applications.DenseNet121(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

def predict_transfer_learning(image):
    img = Image.fromarray(image).resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = transfer_learning_model.predict(img_array)
    predicted_class = labels[np.argmax(prediction)]
    return predicted_class

def predict_cnn(image):
    img = Image.fromarray(image).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = cnn_model.predict(img_array)
    predicted_class = labels[np.argmax(prediction)]
    return predicted_class

def predict_knn(image):
    img = Image.fromarray(image).resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    features = np.reshape(features, (1, 7 * 7 * 1024))
    prediction = knn_model.predict(features)
    predicted_class = labels[prediction[0]]
    return predicted_class

def classify_image(model_choice, image):
    if model_choice == "Transfer Learning":
        return predict_transfer_learning(image)
    elif model_choice == "CNN":
        return predict_cnn(image)
    elif model_choice == "KNN":
        return predict_knn(image)
    else:
        return "Invalid model choice"

# Gradio Interface
interface = gr.Interface(
    fn=classify_image,
    inputs=[
        gr.components.Radio(choices=["Transfer Learning", "CNN", "KNN"], label="Choose Model"),
        gr.components.Image(type="numpy", label="Upload Image")
    ],
    outputs=gr.components.Textbox(label="Predicted Breed")
)

if __name__ == "__main__":
    interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b12d7b84b5a5dfdbb9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
